In [2]:
import pandas as pd
import numpy as np
from gurobipy import *

In [3]:
# cargar la cinética de recuperacion
data_name = 'marvin'
cinetica_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/Daniel_inputs/'
block_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/newman1.blocks'
cinetica_file = 'REC_MODEL_T240.csv'
cinetica_df = pd.read_csv(cinetica_path + cinetica_file)
blocks_df = pd.read_csv(block_path, header=None)
rec = cinetica_df['rec90'] # panda series para la recuperacion en t=90

In [3]:
# histograma de la recuperacion en el instante 90
hist = cinetica_df.hist(column='rec90', bins=5)
# calcular nuevos valores de la funcion objetivo
# usando la recuperación al dia 90
nblocks = blocks_df.shape[0]
# algunos datos dados
c_conv = 2204.62
rec_AU = 0.88
price_AU = 12
seilling_AU = 0.2
rec_CU = 0.75
price_CU = 20
seilling_CU = 7.2
proc_cost = 4
mine_cost = 0.9 # por tonn
bv_list_cinetic = list()
bv_list_calc = list()
economic_value_list = list()
economic_value_rec_list = list()
with open('marvin_cinetica.upit', 'w') as output, open(block_path, 'r') as bm:
    output.write('NAME: marvin\nTYPE: UPIT\nNBLOCKS: 53271\nOBJECTIVE_FUNCTION: \n')
    for i in range(nblocks):
        #recuperar datos de bm para calcular valor de cada bloque
        linea_bm = bm.readline()
        linea_lista = linea_bm.split()
        tonn = float(linea_lista[4])
        AU = float(linea_lista[5])
        CU = float(linea_lista[6])
        economic_value = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec[i]/100*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_rec = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec_CU*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_list.append(economic_value)
        economic_value_rec_list.append(economic_value_rec)

ValueError: could not convert string to float: 'FRWS'

In [146]:
upit_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/marvin.upit'
bv_list = list()
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [21]:
# beneficio con recuperacion de Daniel y calculo posterior (economic value)
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/marvin.prec'
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3052245.7535
Found heuristic solution: objective 2.155104e+07
Found heuristic solution: objective 3.447631e+07
Presolve removed 583953 rows and 46469 columns (presolve time = 5s) ...
Presolve removed 617931 rows and 49095 columns
Presolve time: 6.78s
Presolved: 32700 rows, 4176 columns, 65400 nonzeros
Found heuristic solution: objective 8.477468e+08
Variable types: 0 continuous, 4176 integer (4176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500000e+09   0.000000e+00   4.745000e+03      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e+00      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e

In [173]:
# beneficio con recuperacion de Daniel y calculo posterior (economic value rec cte)
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_rec_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
prec_path = '/Users/administrador2/Desktop/Memoria/Codigos/minelib_inputs/marvin.prec'
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2732201.5030
Found heuristic solution: objective 2.001547e+07
Found heuristic solution: objective 3.193380e+07
Presolve removed 548375 rows and 43446 columns (presolve time = 5s) ...
Presolve removed 587477 rows and 46778 columns (presolve time = 10s) ...
Presolve removed 615087 rows and 48834 columns
Presolve time: 12.87s
Presolved: 35544 rows, 4437 columns, 71088 nonzeros
Found heuristic solution: objective 7.832359e+08
Variable types: 0 continuous, 4437 integer (4437 binary)
Presolve removed 4 rows and 152 columns
Presolved: 4433 rows, 39829 columns, 75225 nonzeros

Presolve removed 4433 rows and 39829 columns

Root simplex log...

Iteration    Objective       Pri

In [74]:
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
data_name = 'zuck_small'
pcpsp_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/' + data_name + '.pcpsp'
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/' + data_name + '.prec'
upit_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/'+ data_name + '.upit'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        if linea_lista[0] == 'NAME:':
            dato = linea_lista[1].strip('\n')
            name = dato
        elif linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    lista = linea.split()
                    resource_constraint_ub_limits[r,t] = int(lista[-1])
                    resource_constraint_lb_limits[r,t] = '-Infinity' # falta hacer para el caso general
    
        elif linea_lista[0] == 'OBJECTIVE_FUNCTION:':
            for b in range(nblocks):
                linea = f.readline()
                lista= linea.split()
                objective_function_pcpsp[b,0] = float(lista[1])
                objective_function_pcpsp[b,1] = float(lista[2])
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_COEFFICIENTS:':
                for linea in f:
                    if linea == 'EOF\n':
                        break
                    lista = linea.split()
                    b = int(lista[0])
                    d = int(lista[1])
                    r = int(lista[2])
                    resource_constraint_coefficients[b,r,d] = float(lista[3])

# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

print('Data base name: %s' % (name))
print('NBLOCKS: %d' % nblocks)
print('NPERIODS: %d' % nperiods)
print('NDESTINATIONS: %d' % ndestinations)
print('NRESOURCE_SIDE_CONSTRAINTS: %d' % nresource_side_constraints)
print('NGENERAL_SIDE_CONSTRAINTS: %d' % ngeneral_side_constraints)
print('DISCOUNT_RATE: %.2f' % discount_rate)

Data base name: Zuck_small
NBLOCKS: 9400
NPERIODS: 20
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.10


In [77]:
bv_list = list() # block value list
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [78]:
# beneficio con valores objetivos dado por .upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(LinExpr([bv_list[i]for i in range(nblocks)], [x[i] for i in range(nblocks)]), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id = [i for i in range(nblocks) if x[i].x==1] # recuperar upit
print('UPIT BLOCKS: %d' % len(blocks_id))

Optimize a model with 145640 rows, 9400 columns and 291280 nonzeros
Variable types: 0 continuous, 9400 integer (9400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 1.060450e+07
Found heuristic solution: objective 1.422727e+09
Presolve removed 66154 rows and 2871 columns
Presolve time: 4.77s
Presolved: 79486 rows, 6529 columns, 158972 nonzeros
Found heuristic solution: objective 7.526241e+08
Variable types: 0 continuous, 6529 integer (6529 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    6581    1.4253184e+09   0.000000e+00   4.390000e+02      5s
    7086    1.4227269e+09   0.000000e+00   0.000000e+00      5s

Root relaxation: cutoff, 7086 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [83]:
# pcpsp model: variables y restricciones
m = Model()
# definir variables para pcpsp
x_pcpsp = {}
y = {}
blocks = blocks_id
for i in blocks:
    for t in range(nperiods):
        x_pcpsp[i,t] = m.addVar(vtype=GRB.BINARY, name = "x_pcpsp%d" % i)
for i in blocks:
    for d in range(ndestinations):
        for t in range(nperiods):
            y[i,d,t] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS)

indices = list(itertools.product(range(nperiods), range(ndestinations),blocks))
objetivo_expr = LinExpr([(1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d] for t,d,b in indices],
                   [y[b,d,t] for t,d,b in indices])
m.setObjective(objetivo_expr, GRB.MAXIMIZE)
#m.setObjective(quicksum([(1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d] * y[b,d,t] for t,d,b in itertools.product(range(nperiods), range(ndestinations),blocks_id)]), GRB.MAXIMIZE)
# restricciones para pcpsp
# resource contraints
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
        resource = LinExpr([resource_constraint_coefficients[b,r,d] for b,d in blockTimesDest], 
                           [y[b,d,t] for b,d in blockTimesDest])
        #m.addConstr(resource_constraint_lb_limits[r,t] <=
        #            quicksum([resource_constraint_coefficients[b,r,d]*y[b,d,t] for b, d in blockTimesDest])
        #            <=resource_constraint_ub_limits[r,t])
        m.addConstr(resource_constraint_lb_limits[r,t] <=
                   resource <= resource_constraint_ub_limits[r,t])
# CliqueBlock
for b in blocks:
    m.addConstr(LinExpr([1 for t in range(nperiods)], [x_pcpsp[b,t] for t in range(nperiods)])<=1)
# SumDest
for b in blocks:
    for t in range(nperiods):
        m.addConstr(LinExpr([1 for d in range(ndestinations)], [y[b,d,t] for d in range(ndestinations)])==x_pcpsp[b,t])
# Precedence
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        b = int(linea_lista[0])
        if b in blocks: # solo agregar precedencia para b in UPIT
            for j in range(nvecinos):
                v = int(linea_lista[j+2])
                for t in range(nperiods):
                    m.addConstr(LinExpr([1 for s in range(t+1)], [x_pcpsp[b,s] for s in range(t+1)]) <= LinExpr( [1 for s in range(t+1)], [x_pcpsp[v,s] for s in range(t+1)]))

In [ ]:
m.optimize()

Optimize a model with 3109579 rows, 563940 columns and 62380400 nonzeros
Variable types: 375960 continuous, 187980 integer (187980 binary)
Coefficient statistics:
  Matrix range     [3e-01, 7e+04]
  Objective range  [1e-03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 90820 columns (presolve time = 15s) ...
Presolve removed 9180 rows and 100000 columns (presolve time = 21s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 25s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 30s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 35s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 40s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 45s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 50s) ...
Presolve removed 90820 rows and 181640 columns (presolve time = 55s) ...
Pre

KeyboardInterrupt: 

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.9750293e+09   0.000000e+00   5.945436e+08    209s


Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Equipo\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 352, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "C:\Users\Equipo\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 190, in schedule
    self._event_pipe.send(event_id)
  File "zmq/backend/cython/socket.pyx", line 636, in zmq.backend.cython.socket.Socket.send (zmq\backend\cython\socket.c:7305)
  File "zmq/backend/cython/socket.pyx", line 683, in zmq.backend.cython.socket.Socket.send (zmq\backend\cython\socket.c:7048)
  File "zmq/backend/cython/socket.pyx", line 201, in zmq.backend.cython.socket._send_copy (zmq\backend\cython\socket.c:2920)
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq\backend\cython\socket.c:9621)
    PyErr_CheckSignals()
KeyboardInterrupt


    9378    3.6580762e+09   0.000000e+00   3.319139e+08    211s
   10359    3.1928085e+09   0.000000e+00   1.234967e+08    215s


...

Exception ignored in: 'gurobipy.logcallbackstub'


   16172    2.4458905e+09   0.000000e+00   9.091184e+06    265s


Traceback (most recent call last):
  File "C:\Users\Equipo\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 343, in write
    if self.pub_thread is None:
KeyboardInterrupt


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Equipo\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 321, in flush
    evt.wait()
  File "C:\Users\Equipo\Anaconda3\lib\threading.py", line 551, in wait
    

In [63]:
for t in range(nperiods):
    blocks_id_schedule = [i for i in blocks_id if x_pcpsp[i,t].x==1]
    print(len(blocks_id_schedule))

406
366
287
0
0
0
